# Libraries Loading

In [9]:
import numpy as np
import pandas as pd
import arabic
import pyarabic
import helpers
from helpers import Clean_data,separate_token_with_dicrites,save_h5
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from numpy import array, argmax
import os,errno,re,sys
import pickle
from itertools import product 
from pyarabic.araby import strip_tashkeel, strip_tatweel
import h5py
import random as rn
##########################
rn.seed(123456)
np.random.seed(123456)
input_data_path = "../data/Almoso3a_Alshe3rya/data/All_ksaied.csv"

In [10]:
try:
    os.makedirs("../data/Encoded/8bits/WithoutTashkeel/Eliminated/")
    os.makedirs("../data/Encoded/8bits/WithoutTashkeel/Full_Data/")
    os.makedirs("../data/Encoded/8bits/WithTashkeel/Eliminated/")
    os.makedirs("../data/Encoded/8bits/WithTashkeel/Full_Data/")
except OSError as e:
    if e.errno != errno.EEXIST:
        print("Can't create file for checkpoints or for logs please check ")
        raise
print("Paths created")

Paths created


# Data Loading

In [37]:
#input_data = pd.read_csv(input_data_path, index_col=0)
len(input_data)

1831770

In [38]:
sample = input_data.sample(1000)
sample

,العصر,الشاعر,الديوان,القافية,البحر,الشطر الايسر,الشطر الايمن,البيت
390449,العباسي,أشجَع السَلمي,الديوان الرئيسي,ر,الطويل,وَلَيـسَ بِـها إِلّا اِنتِقال الصِبا نَفرُ,رَأَيـنـا نِـفـارا مِـن ظِـبـاءٍ أَوانِـسٍ,رَأَيـنـا نِـفـارا مِـن ظِـبـاءٍ أَوانِـسٍ ...
127618,العباسي,العَبّاسِ بنِ الأَحنَف,الديوان الرئيسي,ت,السريع,وَنَـحـنُ نَـشـكو الكُرَبَ الزَجِلات,جَــلَســنَ يَـسـمَـعـنَ أَحـاديـثَـنـا,جَــلَســنَ يَـسـمَـعـنَ أَحـاديـثَـنـا وَن...
1676937,الحديث,أبو المحاسن الكربلائي,الديوان الرئيسي,ف,الرمل,عـرف ظـمـيـاء بـهـا قـد عـرفا,مـن صـبـا نـجـد شـممنا نفحة,مـن صـبـا نـجـد شـممنا نفحة عـرف ظـمـيـاء ب...
570713,الحديث,أحمد محرم,الديوان الرئيسي,ر,الكامل,عَـمـرٌو يُـنَـبِّئـُكِ اليَـقـيـنَ وَيُخبِرُ,لَأَجـابَـكِ الفـاروقُ عَـنّا وَاِنبَرى,لَأَجـابَـكِ الفـاروقُ عَـنّا وَاِنبَرى عَـ...
1637356,الحديث,نبوية موسى,الديوان الرئيسي,ر,البسيط,مِـن شـأنـهـا أن تـسـتبدّ وتقهرا,طـاحَـت بـكـامـلنـا ليـاليـك الّتي,طـاحَـت بـكـامـلنـا ليـاليـك الّتي مِـن شـأ...
424356,الحديث,المفتي فتح الله,الديوان الرئيسي,س,الكامل,يـا مَـنْ رضـيـتُ تـغـرُّبي من أجلِهِ,أَفَــلا تَــرى مــتـغـرِّبـاً عـن أهـلِهِـ,أَفَــلا تَــرى مــتـغـرِّبـاً عـن أهـلِهِـ ...
1686032,الحديث,أحمد الحملاوي,الديوان الرئيسي,ر,البسيط,فــإنــكــم ورســول الله أنـصـارى,بـالله لا تـمـنـعـونـي من رعايتكم,بـالله لا تـمـنـعـونـي من رعايتكم فــإنــكـ...
1734953,الحديث,أسعد طراد,الديوان الرئيسي,ت,&nbsp;,بـالله كـان تـعـيـس مـخلوقاته,والمــرء لولا مــوتـه لحـيـاتـه,والمــرء لولا مــوتـه لحـيـاتـه بـالله كـان...
1777497,الحديث,الأخطل الصغير,الأخطل الصغير,ع,الكامل,ولبـان فـيـه مـذ نـأيـت تـصـدع,جـرس الكـنيسة لو تكلم لاشتكى,جـرس الكـنيسة لو تكلم لاشتكى ولبـان فـيـه م...
1389769,الحديث,محمد سعيد الإسكافي,الديوان الرئيسي,ع,الرمل,NaN,ومــــطــــافــــا للتـــقـــي الأورع,ومــــطــــافــــا للتـــقـــي الأورع


In [62]:
#input_data = pd.read_csv(input_data_path, index_col=0)

# TAHA
#input_data = sample

input_data.info(memory_usage='deep')
bahr = 'Bahr'
bayt = 'Bayt_Text'
#input_data  = input_data[[bayt, bahr]]
bahr = 'البحر'
bayt = 'البيت'
input_data  = input_data[[bayt, bahr]]
input_data.columns = ['Bayt_Text','Bahr']
our_alphabets = "".join(arabic.alphabet) + "".join(arabic.tashkeel)+" "
our_alphabets = "".join(our_alphabets)
input_data['Bayt_Text'] = input_data['Bayt_Text'].apply(lambda x: re.sub(r'[^'+our_alphabets+']','',str(x))).apply(lambda x: re.sub(r'  *'," ",x)).apply(lambda x: re.sub(r'ّ+', 'ّ', x)).apply(lambda x: x.strip())
input_data

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 390449 to 508218
Data columns (total 8 columns):
العصر           1000 non-null object
الشاعر          1000 non-null object
الديوان         1000 non-null object
القافية         1000 non-null object
البحر           1000 non-null object
الشطر الايسر    971 non-null object
الشطر الايمن    1000 non-null object
البيت           1000 non-null object
dtypes: object(8)
memory usage: 1.0 MB


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,Bayt_Text,Bahr
390449,رَأَينا نِفارا مِن ظِباءٍ أَوانِسٍ وَلَيسَ بِه...,الطويل
127618,جَلَسنَ يَسمَعنَ أَحاديثَنا وَنَحنُ نَشكو الكُ...,السريع
1676937,من صبا نجد شممنا نفحة عرف ظمياء بها قد عرفا,الرمل
570713,لَأَجابَكِ الفاروقُ عَنّا وَاِنبَرى عَمرٌو يُن...,الكامل
1637356,طاحَت بكاملنا لياليك الّتي مِن شأنها أن تستبدّ...,البسيط
424356,أَفَلا تَرى متغرِّباً عن أهلِهِ يا مَنْ رضيتُ ...,الكامل
1686032,بالله لا تمنعوني من رعايتكم فإنكم ورسول الله أ...,البسيط
1734953,والمرء لولا موته لحياته بالله كان تعيس مخلوقاته,&nbsp;
1777497,جرس الكنيسة لو تكلم لاشتكى ولبان فيه مذ نأيت تصدع,الكامل
1389769,ومطافا للتقي الأورع,الرمل


In [76]:
# Taha: first class of the first observation
row_first_observation_class = input_data.iloc[0][1]
row_first_observation_class

'الطويل'

# Data Analysis

In [40]:
# Taha: Counts the opservations of each class
input_data.iloc[:, 1].value_counts()

الطويل          226
الكامل          198
البسيط          140
الخفيف           95
الوافر           69
الرجز            65
&nbsp;           38
الرمل            38
السريع           32
المتقارب         30
المنسرح          21
موشح             15
المجتث           10
الهزج             5
المديد            4
المتدارك          4
عامي              3
شعر التفعيلة      2
الدوبيت           2
المواليا          1
شعر حر            1
المضارع           1
Name: Bahr, dtype: int64

# Data Filteration

In [32]:
elminated_classic_bohor = ['الوافر', 'المنسرح','المجتث', 'المتقارب', 'الكامل', 'الطويل', 'السريع', 'الرمل',
                         'الرجز', 'الخفيف', 'البسيط']
elminated_classic_bohor

['الوافر',
 'المنسرح',
 'المجتث',
 'المتقارب',
 'الكامل',
 'الطويل',
 'السريع',
 'الرمل',
 'الرجز',
 'الخفيف',
 'البسيط']

In [33]:
full_bohor_classes = ['الوافر', 'المنسرح', 'المديد', 'المجتث', 'المتقارب', 'الكامل', 'الطويل', 'السريع', 'الرمل',
                         'الرجز', 'الخفيف', 'البسيط', 'المقتضب', 'الهزج', 'المضارع', 'المتدارك']
full_bohor_classes

['الوافر',
 'المنسرح',
 'المديد',
 'المجتث',
 'المتقارب',
 'الكامل',
 'الطويل',
 'السريع',
 'الرمل',
 'الرجز',
 'الخفيف',
 'البسيط',
 'المقتضب',
 'الهزج',
 'المضارع',
 'المتدارك']

In [41]:
full_filtered_data = input_data.loc[input_data['Bahr'].isin(full_bohor_classes)]
full_filtered_data['Bayt_Text'] = full_filtered_data['Bayt_Text'].apply(pyarabic.araby.strip_tatweel)
full_filtered_data['Bayt_Text'] = full_filtered_data['Bayt_Text'].apply(helpers.factor_shadda_tanwin)
#full_filtered_data.head()
# Taha
full_filtered_data.iloc[:, 1].value_counts()

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


الطويل      226
الكامل      198
البسيط      140
الخفيف       95
الوافر       69
الرجز        65
الرمل        38
السريع       32
المتقارب     30
المنسرح      21
المجتث       10
الهزج         5
المديد        4
المتدارك      4
المضارع       1
Name: Bahr, dtype: int64

In [42]:
eliminated_filtered_data = input_data.loc[input_data['Bahr'].isin(elminated_classic_bohor)]
eliminated_filtered_data['Bayt_Text'] = eliminated_filtered_data['Bayt_Text'].apply(pyarabic.araby.strip_tatweel)
eliminated_filtered_data['Bayt_Text'] = eliminated_filtered_data['Bayt_Text'].apply(helpers.factor_shadda_tanwin)
# eliminated_filtered_data.head()

# Taha
eliminated_filtered_data.iloc[:, 1].value_counts()

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


الطويل      226
الكامل      198
البسيط      140
الخفيف       95
الوافر       69
الرجز        65
الرمل        38
السريع       32
المتقارب     30
المنسرح      21
المجتث       10
Name: Bahr, dtype: int64

# Encode Classes

In [47]:
'''
    Taha; Encoding Full Data classes as OneHot vectors. 
'''

full_data_label_encoder = LabelEncoder()
full_data_integer_encoded = full_data_label_encoder.fit_transform(list(full_filtered_data["Bahr"]))
# binary encode
full_data_onehot_encoder = OneHotEncoder(sparse=False)
full_data_integer_encoded = full_data_integer_encoded.reshape(len(full_data_integer_encoded), 1)
full_data_bohor_encoded = full_data_onehot_encoder.fit_transform(full_data_integer_encoded)
save_h5('../data/Encoded/8bits/WithoutTashkeel/Full_Data/full_data_Y_Meters.h5', 'Y', full_data_bohor_encoded)
save_h5('../data/Encoded/8bits/WithTashkeel/Full_Data/full_data_Y_Meters.h5', 'Y', full_data_bohor_encoded)
#saving with pickle: https://stackoverflow.com/questions/11218477/how-can-i-use-pickle-to-save-a-dict
print('saving preprocessors....')
with open("../encoders_full_dat.pickle", 'wb') as pre_file:
    pickle.dump(full_data_label_encoder, pre_file, protocol=pickle.HIGHEST_PROTOCOL)
print('saved ....')

saving preprocessors....
saved ....


In [80]:
### testing
#print('loading saved encoders from %s'%(encoders_file_name))
with open("../encoders_full_dat.pickle", 'rb') as f:
    encoders = pickle.load(f)

##testing 
#inverted = encoders.inverse_transform([argmax(full_filtered_data["Bahr"][1,])])


# Taha
Ys = h5py.File('../data/Encoded/8bits/WithoutTashkeel/Full_Data/full_data_Y_Meters.h5', 'r')
class_of_first_observation = Ys['Y'][0]
decoded_first_class = encoders.inverse_transform([argmax( class_of_first_observation   )])

if decoded_first_class == row_first_observation_class:
    print('NICE, Classes Encoded properly')
else:
    print('There is a problem in encoding-decoding classes.')

NICE, Classes Encoded properly


In [84]:
'''
    Taha; This cell is entirly doublicated.
    It Encodes the eliminated data classes.
'''
eliminated_data_label_encoder = LabelEncoder()
eliminated_data_integer_encoded = eliminated_data_label_encoder.fit_transform(list(eliminated_filtered_data["Bahr"]))
# binary encode
eliminated_data_onehot_encoder = OneHotEncoder(sparse=False)
eliminated_data_integer_encoded = eliminated_data_integer_encoded.reshape(len(eliminated_data_integer_encoded), 1)
eliminated_data_bohor_encoded = eliminated_data_onehot_encoder.fit_transform(eliminated_data_integer_encoded)
save_h5('../data/Encoded/8bits/WithoutTashkeel/Eliminated/Eliminated_data_Y_Meters.h5', 'Y', eliminated_data_bohor_encoded)
save_h5('../data/Encoded/8bits/WithTashkeel/Eliminated/Eliminated_data_Y_Meters.h5', 'Y', eliminated_data_bohor_encoded)
#saving with pickle: https://stackoverflow.com/questions/11218477/how-can-i-use-pickle-to-save-a-dict
print('saving preprocessors....')
with open("../encoders_eliminated_data.pickle", 'wb') as pre_file:
    pickle.dump(eliminated_data_bohor_encoded, pre_file, protocol=pickle.HIGHEST_PROTOCOL)
print('saved ...')

saving preprocessors....
saved ...


# Clean The Data

## WITH TASHKEEL

### FULL DATA

In [86]:
bayt_len_full_withtashkeel = np.max((full_filtered_data.Bayt_Text.apply(separate_token_with_dicrites).apply(len)))
print("bayt_len_full_withtashkeel done..")
full_filtered_data_with_tashkeel_cleaned = Clean_data(data_frame = full_filtered_data,max_bayt_len = bayt_len_full_withtashkeel,verse_column_name = 'Bayt_Text')
print("full_cleaned_data_with_tashkeel cleaned ..")
full_cleaned_data_with_tashkeel_encoded = full_filtered_data_with_tashkeel_cleaned.Bayt_Text.apply(lambda x: helpers.string_with_tashkeel_vectorizer(x, bayt_len_full_withtashkeel))
print("full_cleaned_data_with_tashkeel_encoded encoded ..")
full_cleaned_data_with_tashkeel_encoded_stacked = np.stack(full_cleaned_data_with_tashkeel_encoded,axis=0)
# save('../data/Encoded/8bits/WithTashkeel/Full_Data/full_data_matrix_with_tashkeel_8bitsEncoding.h5', 'X', full_cleaned_data_with_tashkeel_encoded_stacked) 
# Taha
save_h5('../data/Encoded/8bits/WithTashkeel/Full_Data/full_data_matrix_with_tashkeel_8bitsEncoding.h5', 'X', full_cleaned_data_with_tashkeel_encoded_stacked) 
print("full_cleaned_data_with_tashkeel_encoded saved ..")
print(full_cleaned_data_with_tashkeel_encoded_stacked.shape)

bayt_len_full_withtashkeel done..
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268


#### ELIMINATED DATA

In [87]:
bayt_len_eliminated_withtashkeel = np.max((eliminated_filtered_data.Bayt_Text.apply(separate_token_with_dicrites).apply(len)))
print("bayt_len_eliminated_withtashkeel done..")
eliminated_filtered_data_with_tashkeel_cleaned = Clean_data(data_frame = eliminated_filtered_data,max_bayt_len = bayt_len_eliminated_withtashkeel  ,verse_column_name = 'Bayt_Text')
print("eliminated_cleaned_data_with_tashkeel cleaned ..")
eliminated_cleaned_data_with_tashkeel_encoded = eliminated_filtered_data_with_tashkeel_cleaned.Bayt_Text.apply(lambda x: helpers.string_with_tashkeel_vectorizer(x, bayt_len_eliminated_withtashkeel))
print("eliminated_cleaned_data_with_tashkeel_encoded encoded ..")
eliminated_cleaned_data_with_tashkeel_encoded_stacked = np.stack(eliminated_cleaned_data_with_tashkeel_encoded,axis=0)
save_h5('../data/Encoded/8bits/WithTashkeel/Eliminated/eliminated_data_matrix_with_tashkeel_8bitsEncoding.h5', 'X', eliminated_cleaned_data_with_tashkeel_encoded_stacked) 
print("full_cleaned_data_with_tashkeel_encoded saved ..")
print(eliminated_cleaned_data_with_tashkeel_encoded_stacked.shape)
#encoded_byot = data_with_Tashkeel_cleaned.apply(lambda x: helpers.string_with_tashkeel_vectorizer_OneHot(x, maximum_bayt_len))

bayt_len_eliminated_withtashkeel done..
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
26

## WITHOUT TASHKEEL

### FULL DATA

In [88]:
full_filtered_data_without_tashkeel = full_filtered_data['Bayt_Text'].apply(pyarabic.araby.strip_tashkeel)
bayt_len_full_without_tashkeel = np.max((full_filtered_data_without_tashkeel.apply(len)))
print("bayt_len_full_without_tashkeel done..")
full_cleaned_data_without_tashkeel_encoded = full_filtered_data_without_tashkeel.apply(lambda x: helpers.string_with_tashkeel_vectorizer(x, bayt_len_full_without_tashkeel))
print("full_filtered_data_without_tashkeel encoded ..")
full_filtered_data_without_tashkeel_staked = np.stack(full_cleaned_data_without_tashkeel_encoded,axis=0)
save_h5('../data/Encoded/8bits/WithoutTashkeel/Full_Data/full_data_matrix_without_tashkeel_8bitsEncoding.h5', 'X', full_filtered_data_without_tashkeel_staked) 
print("full_filtered_data_without_tashkeel_staked saved ..")
print(full_filtered_data_without_tashkeel_staked.shape)


bayt_len_full_without_tashkeel done..
full_filtered_data_without_tashkeel encoded ..
full_filtered_data_without_tashkeel_staked saved ..
(938, 65, 8)


### ELIMINATE

In [90]:
eliminated_filtered_data_without_tashkeel = eliminated_filtered_data['Bayt_Text'].apply(pyarabic.araby.strip_tashkeel)
bayt_len_eliminated_without_tashkeel = np.max((eliminated_filtered_data_without_tashkeel.apply(len)))
print("bayt_len_eliminated_without_tashkeel done..")
eliminated_cleaned_data_without_tashkeel_encoded = eliminated_filtered_data_without_tashkeel.apply(lambda x: helpers.string_with_tashkeel_vectorizer(x, bayt_len_eliminated_without_tashkeel))
print("eliminated_filtered_data_without_tashkeel encoded ..")
eliminated_filtered_data_without_tashkeel_staked = np.stack(eliminated_cleaned_data_without_tashkeel_encoded,axis=0)
save_h5('../data/Encoded/8bits/WithoutTashkeel/Eliminated/eliminated_data_matrix_without_tashkeel_8bitsEncoding.h5', 'X', full_filtered_data_without_tashkeel_staked) 
print("eliminated_filtered_data_without_tashkeel_staked saved ..")

print(eliminated_filtered_data_without_tashkeel_staked.shape)


bayt_len_eliminated_without_tashkeel done..
eliminated_filtered_data_without_tashkeel encoded ..
eliminated_filtered_data_without_tashkeel_staked saved ..
(924, 65, 8)


## Congratualtions :) Finished :) :) :)